In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from urllib.request import urlretrieve
from scipy.stats import pearsonr

from config.config import DATA_PATH

In [36]:
pd.set_option('display.max_columns', None)

flights = pd.read_csv(DATA_PATH + '/preprocessed/airline_performance.csv', parse_dates=['scheduled_departure_datetime', 'scheduled_arrival_datetime', 'scheduled_departure_datetime_utc', 'scheduled_arrival_datetime_utc'])

In [4]:
flights.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,route_id,hour_of_day,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D,temperature_2m_origin,precipitation_origin,snowfall_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snowfall_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0,3228,23,late night,5.125000,2.0,31.0,-5.000000,-6.0,16.0,3.0,0.0,8.0,15.518519,5.0,82.0,4.923077,0.5,62.0,4.0,0.0,27.0,15.714286,6.0,163.0,4.903614,-2.0,166.0,5.0,0.0,84.0,87.6,0.0,0.0,0.0,0.0,293635.188,5.1,180.0,8.5,72.6,0.0,0.0,0.0,3.0,40026.246,4.2,196.0,8.5
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0,3228,23,late night,14.000000,2.0,76.0,4.714286,-6.0,69.0,3.0,0.0,8.0,17.740741,5.0,82.0,7.961538,5.0,69.0,4.0,0.0,27.0,16.690476,7.0,163.0,5.819277,-1.0,166.0,5.0,0.0,84.0,89.7,0.0,0.0,0.0,0.0,293635.188,13.5,220.0,25.3,70.0,0.0,0.0,0.0,3.0,42322.836,4.9,223.0,18.1
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0,3228,23,late night,12.375000,1.5,76.0,5.000000,-6.0,69.0,3.0,0.0,8.0,15.185185,5.0,82.0,6.192308,0.5,69.0,4.0,0.0,27.0,16.710843,7.0,163.0,5.573171,-2.0,166.0,5.0,0.0,83.0,91.9,0.0,0.0,0.0,3.0,234251.969,11.7,239.0,22.6,72.9,0.0,0.0,0.0,3.0,52821.523,3.4,212.0,9.4
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.

In [5]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1493764 entries, 0 to 1493763
Columns: 104 entries, year to wind_gusts_10m_dest
dtypes: float64(63), int64(21), object(20)
memory usage: 1.2+ GB


In [6]:
flights.shape

(1493764, 104)

In [7]:
flights.columns.to_list()

['year',
 'quarter',
 'month',
 'day_of_month',
 'day_of_week',
 'marketing_airline_id',
 'flight_number_marketing_airline',
 'operating_airline_id',
 'tail_number',
 'origin_airport_id',
 'origin_city_market_id',
 'origin',
 'origin_state',
 'dest_airport_id',
 'dest_city_market_id',
 'dest',
 'dest_state',
 'dep_delay',
 'dep_del15',
 'departure_delay_groups',
 'taxi_out',
 'taxi_in',
 'arr_delay',
 'arr_del15',
 'arrival_delay_groups',
 'cancelled',
 'diverted',
 'scheduled_elapsed_time',
 'actual_elapsed_time',
 'air_time',
 'distance',
 'distance_group',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay',
 'div_airport_landings',
 'code_share_flight',
 'origin_city',
 'destination_city',
 'scheduled_departure_datetime',
 'scheduled_arrival_datetime',
 'actual_departure_datetime',
 'actual_arrival_datetime',
 'overnight_depature',
 'overnight_arrival',
 'airline_mkt',
 'airline_ops',
 'origin_timezone',
 'destination_timezone',
 'scheduled_d

## Columns to Drop


In [37]:
cols_to_drop = ['year', 'marketing_airline_id',  'flight_number_marketing_airline', 'operating_airline_id', 'tail_number',
 'origin_airport_id', 'origin_city_market_id', 'dest_airport_id', 'dest_city_market_id', 'dep_delay', 'dep_del15',
 'departure_delay_groups', 'taxi_out', 'taxi_in', 'arr_delay', 'arr_del15', 'arrival_delay_groups', 'diverted', 'actual_elapsed_time',
 'air_time', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'div_airport_landings',
 'actual_departure_datetime', 'actual_arrival_datetime', 'scheduled_departure_datetime_utc', 'actual_departure_datetime_utc', 
 'scheduled_arrival_datetime_utc', 'actual_arrival_datetime_utc',
 ]

flights = flights.drop(columns=cols_to_drop)

## Missing Values

In [38]:
missing_values = flights.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False).head(60)

,index,count
71,wind_gusts_10m_dest,1317
70,wind_direction_10m_dest,1317
63,temperature_2m_dest,1317
66,snow_depth_dest,1317
67,weather_code_dest,1317
65,snowfall_dest,1317
69,wind_speed_10m_dest,1317
68,visibility_dest,1317
64,precipitation_dest,1317
55,precipitation_origin,708


In [39]:
flights[['cancelled']].sum() / len(flights)

cancelled    0.014416
dtype: float64

In [40]:
flights[flights['wind_gusts_10m_dest'].isna()]['cancelled'].sum()

1

In [41]:
flights_missing_dropped = flights.dropna(subset=['wind_gusts_10m_dest'])
missing_values = flights_missing_dropped.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False).head(60)

,index,count
30,arr_delay_mean_10D,195
31,arr_delay_median_10D,195
32,arr_delay_max_10D,195
28,dep_delay_median_10D,188
29,dep_delay_max_10D,188
27,dep_delay_mean_10D,188
39,arr_delay_mean_30D,173
40,arr_delay_median_30D,173
41,arr_delay_max_30D,173
38,dep_delay_max_30D,166


In [42]:
# calculate the average aggregated performance by airline_mkt and route_id
route_airline_agg = flights_missing_dropped.groupby(['airline_mkt', 'route_id']).agg({
    'dep_delay_mean_10D': 'mean',
    'dep_delay_median_10D': 'mean',
    'dep_delay_max_10D': 'mean',
    'arr_delay_mean_10D': 'mean',
    'arr_delay_median_10D': 'mean',
    'arr_delay_max_10D': 'mean',
    'cancelled_sum_10D': 'mean',
    'div_airport_landings_sum_10D': 'mean',
    'n_flights_10D': 'mean',
    'dep_delay_mean_30D': 'mean',
    'dep_delay_median_30D': 'mean',
    'dep_delay_max_30D': 'mean',
    'arr_delay_mean_30D': 'mean',
    'arr_delay_median_30D': 'mean',
    'arr_delay_max_30D': 'mean',
    'cancelled_sum_30D': 'mean',
    'div_airport_landings_sum_30D': 'mean',
    'n_flights_30D': 'mean',
    'dep_delay_mean_90D': 'mean',
    'dep_delay_median_90D': 'mean',
    'dep_delay_max_90D': 'mean',
    'arr_delay_mean_90D': 'mean',
    'arr_delay_median_90D': 'mean',
    'arr_delay_max_90D': 'mean',
    'cancelled_sum_90D': 'mean',
    'div_airport_landings_sum_90D': 'mean',
    'n_flights_90D': 'mean'
    
}).reset_index()

agg_cols = route_airline_agg.drop(columns=['airline_mkt', 'route_id']).columns.to_list()
agg_cols
# merge the aggregated data back into the missing stats to fill missing values
missing_stats_filled = pd.merge(
    flights_missing_dropped,
    route_airline_agg,
    on=['airline_mkt', 'route_id'],
    suffixes=('', '_agg'),
    how='left'
)

# Fill missing values in the original columns with the aggregated values
for col in agg_cols:
    missing_stats_filled[col] = missing_stats_filled[col].fillna(missing_stats_filled[col + '_agg'])

# Drop the extra columns used for merging
missing_stats_filled = missing_stats_filled.drop(columns=[col + '_agg' for col in agg_cols])
missing_stats_filled.head()

,quarter,month,day_of_month,day_of_week,origin,origin_state,dest,dest_state,cancelled,scheduled_elapsed_time,distance,distance_group,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,is_holiday,route_id,hour_of_day,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D,temperature_2m_origin,precipitation_origin,snowfall_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snowfall_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest
0,3,8,6,7,LAS,NV,CLT,NC,1,263.0,1916.0,8,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,0,3228,23,late night,5.125000,2.0,31.0,-5.000000,-6.0,16.0,3.0,0.0,8.0,15.518519,5.0,82.0,4.923077,0.5,62.0,4.0,0.0,27.0,15.714286,6.0,163.0,4.903614,-2.0,166.0,5.0,0.0,84.0,87.6,0.0,0.0,0.0,0.0,293635.188,5.1,180.0,8.5,72.6,0.0,0.0,0.0,3.0,40026.246,4.2,196.0,8.5
1,3,8,7,1,LAS,NV,CLT,NC,0,263.0,1916.0,8,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,0,3228,23,late night,14.000000,2.0,76.0,4.714286,-6.0,69.0,3.0,0.0,8.0,17.740741,5.0,82.0,7.961538,5.0,69.0,4.0,0.0,27.0,16.690476,7.0,163.0,5.819277,-1.0,166.0,5.0,0.0,84.0,89.7,0.0,0.0,0.0,0.0,293635.188,13.5,220.0,25.3,70.0,0.0,0.0,0.0,3.0,42322.836,4.9,223.0,18.1
2,3,8,9,3,LAS,NV,CLT,NC,0,258.0,1916.0,8,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,0,3228,23,late night,12.375000,1.5,76.0,5.000000,-6.0,69.0,3.0,0.0,8.0,15.185185,5.0,82.0,6.192308,0.5,69.0,4.0,0.0,27.0,16.710843,7.0,163.0,5.573171,-2.0,166.0,5.0,0.0,83.0,91.9,0.0,0.0,0.0,3.0,234251.969,11.7,239.0,22.6,72.9,0.0,0.0,0.0,3.0,52821.523,3.4,212.0,9.4
3,3,8,10,4,LAS,NV,CLT,NC,0,258.0,1916.0,8,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,0,3228,23,late night,10.111111,1.0,76.0,0.625000,-8.5,69.0,2.0,0.0,9.0,11.851852,3.0,76.0,2.653846,-0.5,69.0,4.0,0.0,27.0,16.120482,7.0,163.0,4.573171,-3.0,166.0,5.0,0.0,83.0,87.6,0.0,0.0,0.0,0.0,173884.516,5.9,18.0,9.4,68.7,0.0,0.0,0.0,0.0,44947.508,1.1,90.0,1.6
4,3,8,12,6,LAS,NV,CLT,NC,0,258.0,1916.0,8,0,Las Vegas,Charlotte,2023-08-12 23:10:00,2023-08-13 06:28:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,0,3228,23,late night,12.250000,3.0,76.0,2.500000,-5.0,69.0,1.0,0.0,8.0,12.961538,4.5,76.0,4.080000,0.0,69.0,4.0,0.0,26.0,15.682927,5.0,163.0,4.098765,-4.0,166.0,5.0,0.0,82.0,92.2,0.0,0.0,0.0,1.0,265091.875,10.2,50.0,13.0,69.0,0.0,0.0,0.0,0.0,40354.332,3.1,176.0,6.5


In [43]:
missing_values = missing_stats_filled.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False).head(60)

,index,count
9,scheduled_elapsed_time,1


In [46]:
missing_time = missing_stats_filled[missing_stats_filled['scheduled_elapsed_time'].isna()]
missing_time['scheduled_elapsed_time'] = (pd.to_datetime(missing_time['scheduled_arrival_datetime']) - pd.to_datetime(missing_time['scheduled_departure_datetime'])).dt.total_seconds() / 60
missing_time

/var/folders/b2/nkpmrw0s42q_zm32rwshljyr0000gn/T/ipykernel_61486/1857774745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_time['scheduled_elapsed_time'] = (pd.to_datetime(missing_time['scheduled_arrival_datetime']) - pd.to_datetime(missing_time['scheduled_departure_datetime'])).dt.total_seconds() / 60


,quarter,month,day_of_month,day_of_week,origin,origin_state,dest,dest_state,cancelled,scheduled_elapsed_time,distance,distance_group,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,is_holiday,route_id,hour_of_day,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D,temperature_2m_origin,precipitation_origin,snowfall_origin,snow_depth_origin,weather_code_origin,visibility_origin,wind_speed_10m_origin,wind_direction_10m_origin,wind_gusts_10m_origin,temperature_2m_dest,precipitation_dest,snowfall_dest,snow_depth_dest,weather_code_dest,visibility_dest,wind_speed_10m_dest,wind_direction_10m_dest,wind_gusts_10m_dest
1420559,1,3,19,7,SFO,CA,LAX,CA,1,89.0,337.0,2,1,San Francisco,Los Angeles,2023-03-19 10:20:00,2023-03-19 11:49:00,0,0,Alaska Airlines Inc.,SkyWest Airlines Inc.,America/Los_Angeles,America/Los_Angeles,0,5963,10,morning,4.296296,-2.0,69.0,-3.407407,-8.0,67.0,3.0,0.0,27.0,7.341176,-3.0,342.0,-2.752941,-12.0,314.0,4.0,0.0,85.0,5.577273,-4.0,383.0,-2.631818,-11.0,369.0,6.0,0.0,220.0,50.8,0.0,0.0,0.0,3.0,43635.172,2.7,90.0,11.6,59.6,0.0,0.0,0.0,3.0,67585.305,4.8,221.0,11.4


In [48]:
missing_stats_filled['scheduled_elapsed_time'] = (pd.to_datetime(missing_stats_filled['scheduled_arrival_datetime']) - pd.to_datetime(missing_stats_filled['scheduled_departure_datetime'])).dt.total_seconds() / 60

In [49]:
missing_values = missing_stats_filled.isna().sum().reset_index(name='count')
missing_values[missing_values['count'] > 0].sort_values('count', ascending=False)

,index,count
